In [1]:
import pandas as pd
from tqdm import tqdm
import gensim
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.chdir(r"D://Proposal_Defense//Simulations")
# from Utils.Script_utils import get_data_splits, first_LSTM_training, get_data_splits_old_algo
from joblib import Parallel, delayed
import sys
from scipy.stats import norm

In [2]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# # 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")
# tf.config.set_visible_devices([], 'GPU')

from tqdm.notebook import tqdm
tf.random.set_seed(seed_value)
import multiprocessing
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
# from keras import backend as K
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# K.set_session(sess)

No GPU found


In [3]:
import matplotlib.pyplot as plt

import pickle

In [4]:
# data["high_level_substr"].value_counts()

In [5]:
# alginate_doc2vec_vecs

In [6]:
# data["high_level_substr"].value_counts()

In [7]:
path = os.getcwd() + "//dataset_1"

In [8]:
# ## read in all the drug features
# ## for train test and valid
train_drugs = np.load(path + "//" + "train_drug.npy")
# valid_drugs = np.load(path + "//" + "valid_drug.npy")
# test_drugs = np.load(path + "//" + "test_drug.npy")

In [9]:
# ## read in all the omics features
# ## for train test and valid
train_omics = np.load(path + "//" + "train_omics.npy")[:,:30]
# valid_omics = np.load(path + "//" + "valid_omics.npy")
# test_omics = np.load(path + "//" + "test_omics.npy")

In [10]:
# ## read in all the labels 
# ## for train test and valid
# y_train = np.load(path + "//" + "train_y.npy")
# y_valid = np.load(path + "//" + "valid_y.npy")
# y_test = np.load(path + "//" + "test_y.npy")

In [11]:
# train_drugs.shape

In [12]:
# valid_drugs.shape

In [13]:
# test_drugs.shape

In [14]:
# y_train.shape

In [15]:
# y_valid.shape

In [16]:
# y_test.shape

In [17]:
def ann(hidden = 10): 
    input_layer = tf.keras.layers.Input(shape = (X_train_word2vec.shape[1]))
    hidden_layer = tf.keras.layers.Dense(hidden)
    hidden_output = hidden_layer(input_layer)
    pred_layer = tf.keras.layers.Dense(1)
    pred_output = pred_layer(hidden_output)
#     pred_output = tf.keras.layers.Activation("softmax")(pred_output)
    model = tf.keras.models.Model(input_layer, pred_output)
    return model

In [18]:
X_train_word2vec = train_omics

In [19]:
h1, h2 = 16,16

In [20]:


ann_15 = ann(h1)


# In[35]:


# ann_15.summary()


# In[36]:



In [21]:
ann_15.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 dense (Dense)               (None, 16)                496       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


In [22]:
X_train_word2vec = train_drugs

In [23]:
ann_20 = ann(h2)



# ann_20.summary()



In [24]:
total_weights_1 = ann_15.count_params()

In [25]:
total_weights_2 = ann_20.count_params()

In [26]:
total_weights = total_weights_1 + total_weights_2 + 1 

In [27]:
total_weights

1987

In [28]:
## batch size
batch_size = 128

In [29]:
# var_weights = 0.08
# var_targets = 0.08

In [30]:
from scipy.stats import multivariate_normal as mvn

In [31]:
from scipy.stats import invgamma, norm

In [32]:
from tqdm.notebook import tqdm

In [33]:
threshold = 5

In [34]:
import random

In [35]:
lr = 0.1

In [36]:
import numpy as np

In [37]:
weights_ann_1 = ann_15.get_weights()
weights_ann_2 = ann_20.get_weights()

In [38]:
def get_targets_with_weights(batch_data, batch_data1, initial_ensembles, log_sigma_points_1): 

    n_hidden_1 = len(weights_ann_1[0].ravel())

    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)

    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)



    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, 1)

    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + 1)].reshape(size_ens, 1, 1)



    final_output_1 = output_1 + output_layer_bias_1

    n_hidden_2 = len(weights_ann_2[0].ravel())

    initial_ensembles_1 = initial_ensembles.copy()[:, total_weights_1:(total_weights_1+ total_weights_2)]

    hidden_weights_2 = initial_ensembles_1[:,:n_hidden_2].reshape(size_ens, batch_data1.shape[1], h2)



    hidden_output_2 = np.einsum('ij,kjl->kil', batch_data1, hidden_weights_2)

    hidden_layer_bias_2 = initial_ensembles[:,n_hidden_2:(n_hidden_2 + h2)].reshape(size_ens, 1,  h2)

    hidden_output_2 = hidden_output_2+ hidden_layer_bias_2

    n_pred_weights_2 = len(weights_ann_2[2].ravel())

    output_weights_2 = initial_ensembles_1[:,(n_hidden_2 + h2):(n_hidden_2 + h2 + n_pred_weights_2) ].reshape(size_ens, h2, 1)


    output_2 = np.einsum('ijk,ikl->ijl', hidden_output_2, output_weights_2)


    output_layer_bias_2 = initial_ensembles_1[:,(n_hidden_2 + h2 + n_pred_weights_2):(n_hidden_2 + h2 + n_pred_weights_2 + 1)].reshape(size_ens, 1, 1)


    final_output_2 = output_2 + output_layer_bias_2


    weights_1 = initial_ensembles[:, :total_weights_1]

    weights_2 = initial_ensembles[:, total_weights_1:(total_weights_1 + total_weights_2)]


    avg_weights = initial_ensembles[:, -1].reshape(-1,1)

    avg_weights_sig = expit(avg_weights)
    
    avg_weights_sig = avg_weights_sig.reshape(avg_weights_sig.shape[0], 1, avg_weights_sig.shape[1])
    
    complement_weights_sig = 1 - expit(avg_weights)
    
    complement_weights_sig = complement_weights_sig.reshape(complement_weights_sig.shape[0], 1, complement_weights_sig.shape[1])

    final_output_1 = final_output_1*complement_weights_sig
    
    final_output_2 = final_output_2*avg_weights_sig
    
    output_1_ravel = final_output_1.reshape(size_ens, final_output_1.shape[1]*final_output_1.shape[2])

    output_2_ravel = final_output_2.reshape(size_ens, final_output_2.shape[1]*final_output_2.shape[2])


    output_1_ravel = output_1_ravel

    output_2_ravel = output_2_ravel



    weights_1_add = np.zeros((size_ens, (total_weights_2 - total_weights_1)))



    weights_1 = np.hstack((weights_1, weights_1_add))
    


    stack_1 = np.hstack((output_1_ravel, weights_1, np.repeat(0, size_ens).reshape(-1,1), np.repeat(0, size_ens).reshape(-1,1)))



    
    stack_2 = np.hstack((output_2_ravel, weights_2, avg_weights, log_sigma_points_1))

    
    initial_aug_state = np.hstack((stack_1, stack_2)) 
    

    return initial_aug_state , output_1_ravel, output_2_ravel, log_sigma_points_1



In [39]:
import time

In [40]:
from scipy.stats import multivariate_normal as mvn

In [41]:
var_weights =0.5
# var_weights_vec = 4
# var_targets = 0.04
# var_weights = 2

In [42]:
from scipy.stats import multivariate_normal as mvn

In [43]:
reduction = 6
# reduction = 10

In [44]:
shape_needed = (total_weights + 2*batch_size*1 + 1 + (total_weights_2 - total_weights_1))//reduction

In [45]:
size_ens = shape_needed

In [46]:
size_ens = int(size_ens)

In [47]:
size_ens

534

In [48]:
# train_fudged_probs

In [49]:
import matplotlib.pyplot as plt

In [50]:
from scipy.special import expit

In [51]:
# catch1[0][0]

In [52]:
# catch_train_logits_second[0]

In [53]:
from datetime import datetime

In [54]:
def expit(x):
    """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
    return 1 / (1 + np.exp(-x))

In [55]:
from scipy.stats import gamma

In [56]:
# second_lstm[0]

In [57]:
# reduction = 8
# var_weights = 1

In [58]:
# cur_idx = 16
# np.log(catch_test_probs[cur_idx]/(1-catch_test_probs[cur_idx]))

In [59]:
# catch_preds_df

In [60]:
# ids = [i for i in range(0, catch_test_probs[cur_idx].shape[0]) if catch_test_probs[cur_idx][i] < 0.5]
# ids_more = [i for i in range(0, catch_test_probs[cur_idx].shape[0]) if catch_test_probs[cur_idx][i] > 0.5]

In [61]:
# ids_more

In [62]:
import plotly.express as px

In [63]:
import seaborn as sns

In [64]:
use_cores = multiprocessing.cpu_count()-1

In [65]:
use_cores = 15

In [66]:
# with open('first_lstm_with_doc2vec_lstm_var_0.0001_real_world.pkl', 'rb') as f:
#     first_lstm = pickle.load(f)  

In [67]:
# threshold = 200
# cutoff_threshold = 70

In [68]:
# catch1 = first_lstm

In [69]:
# catch1[0]

In [70]:
from datetime import datetime

In [71]:
# y_train.shape

In [72]:
# batch_size = 64

In [73]:
from sklearn.metrics import mean_squared_error

In [74]:
import random

In [75]:
gamma_param = 500

In [76]:
def rep_one_real_world(idx, inflation_factor = 0.2, cutoff = 100): 
    catch_1 = []
    catch_2 = []
    catch_3 = []
    catch_4 = []
    catch_5 = []
#     from scipy.special import expit
    patience_smaller = 0
    patience_uns = 0
# patience_bigger = 0

#     best_train_acc = 0
#     best_valid_acc = 1000

#     best_valid_mae = 10
    
    best_train_width = 100000
    
    path = os.getcwd() + "//dataset_" + str(idx + 1) + "/"

    ## read in all the drug features
    ## for train test and valid
    train_drugs = np.load(path + "//" + "train_drug.npy")
    valid_drugs = np.load(path + "//" + "valid_drug.npy")
    test_drugs = np.load(path + "//" + "test_drug.npy")

    ## read in all the omics features
    ## for train test and valid
    train_omics = np.load(path + "//" + "train_omics.npy")[:,:30]
    valid_omics = np.load(path + "//" + "valid_omics.npy")[:,:30]
    test_omics = np.load(path + "//" + "test_omics.npy")[:,:30]

    ## read in all the labels 
    ## for train test and valid
    y_train = np.load(path + "//" + "train_y.npy")
    y_valid = np.load(path + "//" + "valid_y.npy")
    y_test = np.load(path + "//" + "test_y.npy")
    
    X_train_logits = y_train.reshape(-1,1)
#     X_train_logits = catch_train_logits_second[idx]
    
    X_valid_logits = y_valid.reshape(-1,1)
    X_test_logits = y_test.reshape(-1,1)
    
    ## create training batch chunks
    train_idx = list(range(0, X_train_logits.shape[0]))
    batch_chunks = [train_idx[i:i+batch_size] for i in range(0,len(train_idx),batch_size)]

    ## generate some augmented variable for iteration 0
    initial_aug_state_mean = np.repeat(0, total_weights)
    initial_aug_state_mean = initial_aug_state_mean.reshape(-1,1)

    initial_aug_state_cov = var_weights*np.identity((total_weights))
    initial_ensembles = mvn(initial_aug_state_mean.reshape(initial_aug_state_mean.shape[0],), initial_aug_state_cov).rvs(size = size_ens)

    exit_iter_no_thresh = 0
    log_sigma_points_1 = (np.log(gamma(gamma_param, scale = 1/100).rvs(size_ens))).reshape(size_ens, 1)
    

#     y_train = catch_train_labels_second[idx]

#     y_valid = catch_valid_labels_second[idx]

#     y_test = catch_test_labels_second[idx]
    
    
    train_lstm = train_omics
    valid_lstm = valid_omics
    # valid_word2vec = catch[idx][7].iloc[catch_valid_idxes[idx],:].reset_index(drop = True)
    test_lstm = test_omics
 

    train_doc2vec = train_drugs
    valid_doc2vec = valid_drugs
    # valid_word2vec = catch[idx][7].iloc[catch_valid_idxes[idx],:].reset_index(drop = True)
    test_doc2vec = test_drugs
    
    
    
    # train_valid_test_lstm = np.vstack((train_lstm, valid_lstm, test_lstm))
    # train_valid_test_doc2vec = np.vstack((train_doc2vec, valid_doc2vec, test_doc2vec))
    
    # train_valid_test_lstm = np.vstack((train_lstm, valid_lstm))
    # train_valid_test_doc2vec = np.vstack((train_doc2vec, valid_doc2vec))    
    
    train_valid_lstm = (train_lstm)
    train_valid_doc2vec = (train_doc2vec)

    # best_width_train = 100
    
    # threshold_achieved = False
    # satisfactory = False
    # satisfactory_counter = 0
    
    
    best_coverage_train = 0
    
    start = datetime.now()
    
    for iter1 in range(0,500):
        random_idx = random.sample(range(train_valid_lstm.shape[0]), train_valid_lstm.shape[0])
        train_valid_lstm =train_valid_lstm[random_idx, :]
        train_valid_doc2vec = train_valid_doc2vec[random_idx, :]

        for batch_idx in (batch_chunks):

            batch_data = train_valid_lstm[batch_idx,:]
            batch_data1 = train_valid_doc2vec[batch_idx,:]
            # print(batch_data.shape)
            batch_targets = X_train_logits[batch_idx,:]
            
            # print(batch_targets.shape)
            
            # batch_targets = batch_targets.ravel().reshape(-1,1)

            column_mod_2_shape = total_weights_2 + batch_data.shape[0]*1 + 1 + 1
        
            H_t = np.hstack((np.identity(batch_targets.shape[0]), np.zeros((batch_targets.shape[0], column_mod_2_shape-batch_targets.shape[0]))))

            current_aug_state, column_mod_1, column_mod_2, log_sigma_points_1 = get_targets_with_weights(batch_data, batch_data1, initial_ensembles, log_sigma_points_1)
            
            # print(current_aug_state.shape)
            
            
            var_targets_vec = np.log(1 + np.exp(log_sigma_points_1))
            
            var_targets_vec = var_targets_vec
            
            # current_aug_state_var = np.cov(current_aug_state.T) + inflation_factor*np.identity(current_aug_state.shape[1])
            
            current_aug_state_var = np.cov(current_aug_state.T) 
            
            G_t = np.array([1 , 1]).reshape(-1,1)
            
            scirpt_H_t = np.kron(G_t.T, H_t)
            
            temp1 = current_aug_state_var@scirpt_H_t.T
            
            temp2 = scirpt_H_t@current_aug_state_var@scirpt_H_t.T
        
            for ensemble_idx in range(0, current_aug_state.shape[0]):
                
                var_targets1 = var_targets_vec[ensemble_idx,:]
                
                R_t = var_targets1*np.identity(batch_targets.shape[0])
            
                measurement_error = mvn(np.repeat(0,batch_targets.shape[0]), var_targets1*np.identity(batch_targets.shape[0])).rvs(1).reshape(-1,1)
            
                target_current = batch_targets + measurement_error
                
                # print(target_current.shape)
                
                K_t = temp1@np.linalg.inv(temp2 + R_t)
                
                # print(K_t.shape)

                current_aug_state[ensemble_idx,:] = current_aug_state[ensemble_idx,:] +(K_t@(target_current -scirpt_H_t@current_aug_state[ensemble_idx,:].reshape(-1,1))).reshape(current_aug_state.shape[1],)
        

            weights_ann_1 = current_aug_state[:,batch_targets.shape[0]:(batch_targets.shape[0] + total_weights_1)]      

            weights_ann_2 = current_aug_state[:,-(total_weights_2+1):-2]    

            initial_ensembles = np.hstack((weights_ann_1, weights_ann_2, current_aug_state[:,-2].reshape(-1,1)))
            
            log_sigma_points_1 = current_aug_state[:,-1].reshape(-1,1)
               
            avg_betas = expit(current_aug_state[:,-2])
        
            complement = 1-avg_betas

            
            
            
            
            current_aug_state1, column_mod_11, column_mod_21, log_sigma_points_1 = get_targets_with_weights(valid_lstm, valid_doc2vec, initial_ensembles, log_sigma_points_1)
            
            initial_targets = column_mod_11 + column_mod_21
            
            
            initial_targets = initial_targets.reshape(size_ens, valid_lstm.shape[0],1)
            
            initial_targets_train = initial_targets
            
            
            
            
#             ind = (X_train_logits_true >= np.percentile(initial_targets_train, axis = 0, q = (2.5, 97.5))[0,:,:]) & (X_train_logits_true <= np.percentile(initial_targets_train, axis = 0, q = (2.5, 97.5))[1,:,:])
        
            # initial_targets_softmax = expit(initial_targets)
        
            # initial_softmax_train = initial_targets_softmax
            
            initial_targets_train_mean = initial_targets_train.mean(0)
            
            li = np.percentile(initial_targets_train, axis = 0, q = (2.5, 97.5))[0,:,:]
            
            ui = np.percentile(initial_targets_train, axis = 0, q = (2.5, 97.5))[1,:,:]
            
            width = ui - li
            
            avg_width_train = np.mean(width)
            
            # interim = np.vstack((catch_train_probs[idx] , catch_valid_probs[idx] ))
        
            interim = (X_valid_logits)
            
            ind = (interim >= li) & (interim <= ui)
            
            coverage_train= np.mean(ind.ravel())  
            
            train_mae = np.sqrt(mean_squared_error(interim, initial_targets_train_mean))
            # train_mae = np.sqrt(np.mean(np.abs(interim.ravel() - initial_targets_train_mean.ravel())**2))
        
            current_aug_state1, column_mod_11, column_mod_21, log_sigma_points_1 = get_targets_with_weights(test_lstm, test_doc2vec, initial_ensembles, log_sigma_points_1)
            
            initial_targets = column_mod_11 + column_mod_21
            
            initial_targets = initial_targets.reshape(size_ens, test_lstm.shape[0],1)
            
            initial_targets_test = initial_targets
            
            # initial_targets_softmax = expit(initial_targets)    
            
            initial_targets_test_mean = initial_targets_test.mean(0)
            
            li = np.percentile(initial_targets_test, axis = 0, q = (2.5, 97.5))[0,:,:]
            
            ui = np.percentile(initial_targets_test, axis = 0, q = (2.5, 97.5))[1,:,:]
            
            width = ui - li
            
            avg_width = np.mean(width)
            
            catch_test_probs = (X_test_logits)
            
            ind_test = (catch_test_probs >= li) & (catch_test_probs <= ui)
               

                        
            coverage_test = np.mean(ind_test.ravel())    

            # initial_targets_test_mean = initial_targets_train.mean(0)
    
            
            test_mae = np.sqrt(mean_squared_error(catch_test_probs, initial_targets_test_mean))
            
            # test_mae = np.sqrt(np.mean(np.abs(catch_test_probs.ravel() - initial_targets_test_mean.ravel())**2))
       
        # print(avg_width_train, best_train_width, coverage_train)
        
#         print("epoch "+ str(iter1))
#         print("patience "+ str(patience_smaller))
#         print("train mae is " + str(train_mae))
#         print("test mae is " + str(test_mae))
        
#         print("train coverage is "+ str(coverage_train))
#         print("train width is " + str(avg_width_train))        
#         print("test coverage is "+ str(coverage_test))
#         print("test width is " + str(avg_width))
#         print("drugs weight is " + str(np.mean(complement)))
        # print("threshold " + str(threshold_achieved))
              
        
        if iter1 == 0: 
            best_train_mae = train_mae
            best_test_mae = test_mae
            exit_iter_no_thresh = iter1
            best_test_preds = initial_targets_test
            patience_smaller = 0
            best_train_coverage = coverage_train
            best_test_coverage = coverage_test
            best_train_width = avg_width_train
            best_test_width = avg_width
            best_drugs_weight = np.mean(complement)
        
        
        if (avg_width_train < best_train_width) & (coverage_train > 0.75): 
#             cur_best_train_width = avg_width_train
#             cur_best_test_width = avg_width

#             cur_best_train_coverage = coverage_train
#             cur_best_test_coverage = coverage_test 
#             cur_best_lstm_weight = np.mean(complement)
            best_train_mae = train_mae
            best_test_mae = test_mae
            exit_iter_no_thresh = iter1
            best_test_preds = initial_targets_test
            patience_smaller = 0
            best_train_coverage = coverage_train
            best_test_coverage = coverage_test
            best_train_width = avg_width_train
            best_test_width = avg_width
            best_drugs_weight = np.mean(complement)
            # satisfactory = True
            
        else:
            patience_smaller+=1
            
        if patience_smaller > threshold:
            
            break
            
        if (coverage_train < 0.4): 
            patience_uns+=1
            
        if (patience_uns > 3):            
            
            break
            
#     base_model = tf.keras.models.load_model("Real_World_LSTM_Models" + "//" + "Model_" + str(idx))
#     extract_model = tf.keras.models.Model(base_model.input, base_model.layers[-3].output)
#     alginate_lstm_embs = extract_model(np.array(alginate_seqs))
#     current_aug_state1, column_mod_11, column_mod_21, log_sigma_points_1 = get_targets_with_weights(alginate_lstm_embs, alginate_doc2vec_vecs, initial_ensembles, log_sigma_points_1)
            
#     initial_targets = column_mod_11 + column_mod_21
            
#     initial_targets = initial_targets.reshape(size_ens, alginate_doc2vec_vecs.shape[0],1)
            
#             # initial_targets_test = initial_targets
            
#     initial_targets_softmax = expit(initial_targets)    
            
#             # initial_targets_test_mean = initial_targets_softmax.mean(0)
            
#     li = np.percentile(initial_targets_softmax, axis = 0, q = (2.5, 97.5))[0,:,:]
            
#     ui = np.percentile(initial_targets_softmax, axis = 0, q = (2.5, 97.5))[1,:,:]
            
#     width = ui - li
            
#     avg_width_alginate = np.mean(width)                
            
            
    print(best_test_mae, best_test_coverage, best_test_width, flush = True)
    end = datetime.now()
    total = (end- start)
    time_taken = total.seconds/60.0
    return best_train_mae, best_test_mae, best_train_coverage, best_train_width, best_test_coverage, best_test_width,  best_drugs_weight, exit_iter_no_thresh, time_taken, best_test_preds
        
    # print("something went awry", flush = True)

In [77]:
threshold = 30

In [78]:
cutoff_threshold = 30

In [79]:
# plt.hist(y_train)
# plt.show()

In [80]:
batch_size = 32

In [81]:
best_train_mae, best_test_mae, best_train_coverage, best_train_width, best_test_coverage, best_test_width,  best_drugs_weight, exit_iter_no_thresh, time_taken, best_test_preds = rep_one_real_world(1)

2.7008890313290297 0.99 12.849709672732194


In [82]:
# path = r"/lustre/work/statgrads/vpiyush2/enkf_simulations/Old_Strategy_EnKF_LSTM_Doc2Vec_Simulations/DeepCDR/dataset_" + str(1 + 1) + "/"
# y_test = np.load(path + "//" + "test_y.npy")
    

In [83]:
# best_test_mae

In [84]:
# np.sqrt(mean_squared_error(best_test_preds.mean(0),  y_test.reshape(-1,1)))

In [85]:
# best_train_mae, best_test_mae, best_train_coverage, best_train_width, best_test_coverage,  best_test_width

In [86]:
# best_test_preds.shape

In [87]:
# np.sqrt(np.mean(y_test.reshape(-1,1)- best_test_preds.mean(0))**2)

In [88]:
# best_test_preds.mean(0).shape

In [89]:
# plt.scatter(y_test.reshape(-1,1), best_test_preds.mean(0))

In [90]:
# np.sqrt(np.mean((y_test-best_test_preds.mean(0))**2))

In [91]:
# best_test_preds.mean(0)

In [92]:
catch_coverages = Parallel(n_jobs=10, verbose = 10, backend = "loky")(delayed(rep_one_real_world)(i) for i in range(0,10))

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


2.4631526130469177 0.99 13.826083302636633
2.930730772235116 0.93 10.293535434416288
2.804955039126143 0.93 10.327234638932463


[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  2.0min remaining:  4.7min


2.31147179146448 0.95 9.601683169840221
2.2813092250787625 1.0 13.147938111967898


[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  2.1min remaining:  2.1min


2.11465577933989 0.97 11.59590749017658
3.1259858194128234 0.98 13.651759704435358
2.1084743991866377 0.78 6.041518925193459


[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  2.5min remaining:  1.1min


2.293299289360535 0.66 3.9766436569764476
1.8145593252498566 0.84 5.8240456610150275


[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  3.2min finished


In [93]:
# string = "Real_World_EnKF_DeepCDR_" + "Neurons_" + str(h1)  + "_Gamma_" +  str(gamma_param) +  "_var_weights_" + str(var_weights) + "_num_ens_" + str(size_ens)

In [94]:
string = "Real_World_EnKF_DeepCDR_" + "Neurons_" + str(h1)  + "_Gamma_" +  str(gamma_param) +  "_var_weights_" + str(var_weights) + "_num_ens_" + str(size_ens)

In [95]:
string

'Real_World_EnKF_DeepCDR_Neurons_16_Gamma_500_var_weights_0.5_num_ens_534'

In [96]:
with open(string + ".pkl", 'wb') as f:
    pickle.dump(catch_coverages, f)

In [97]:
# catch_coverages

In [98]:
check = pd.DataFrame(catch_coverages).iloc[:,:-1]

In [99]:
# check

In [100]:
# check

In [101]:
# check.columns = ["train_mae", "test_mae", "avg_ci_width_train", "avg_ci_width_test", "avg_lstm_weight", "exit_iter", "exit_status"]

In [102]:
check

,0,1,2,3,4,5,6,7,8
0,2.409395,1.814559,0.80,6.305636,0.84,5.824046,0.509393,2,3.083333
1,2.467920,2.114656,0.87,9.334234,0.97,11.595907,0.540132,0,2.066667
2,2.619765,2.804955,0.87,9.057397,0.93,10.327235,0.454207,0,1.833333
3,2.311016,2.293299,0.84,5.208020,0.66,3.976644,0.494393,1,2.500000
4,2.824174,3.125986,1.00,11.783717,0.98,13.651760,0.479499,0,2.333333
5,2.795989,2.463153,0.94,12.033811,0.99,13.826083,0.487724,0,1.716667
6,2.354518,2.311472,0.94,9.330114,0.95,9.601683,0.384552,0,1.950000
7,2.491844,2.930731,0.94,9.126487,0.93,10.293535,0.541473,0,1.833333
8,2.829252,2.281309,0.98,12.414253,1.00,13.147938,0.484440,0,1.983333
9,2.127707,2.108474,0.83,5.486479,0.78,6.041519,0.499796,1,2.333333


In [103]:
# check.columns = ["train_mae", "test_mae", "train_coverage", "train_width", "test_coverage", "test_width", "exit_iter", "time_taken", "avg_width_alginate"]

In [104]:
check.columns = ["train_mae", "test_mae", "train_coverage", "train_width", "test_coverage", "test_width", "drugs_weight", "exit_iter", "time_taken"]

In [105]:
check.mean()

train_mae         2.523158
test_mae          2.424859
train_coverage    0.901000
train_width       9.008015
test_coverage     0.903000
test_width        9.828635
drugs_weight      0.487561
exit_iter         0.400000
time_taken        2.163333
dtype: float64

In [106]:
check.median()

train_mae          2.479882
test_mae           2.302386
train_coverage     0.905000
train_width        9.228300
test_coverage      0.940000
test_width        10.310385
drugs_weight       0.491059
exit_iter          0.000000
time_taken         2.025000
dtype: float64

In [107]:
check.mean()

train_mae         2.523158
test_mae          2.424859
train_coverage    0.901000
train_width       9.008015
test_coverage     0.903000
test_width        9.828635
drugs_weight      0.487561
exit_iter         0.400000
time_taken        2.163333
dtype: float64

In [108]:
check_mean = check.mean().reset_index()

In [109]:
check_mean.to_csv("Mean_Metrics_" +  string +  ".csv", index = False)

In [110]:
check.std()

train_mae         0.239182
test_mae          0.410012
train_coverage    0.067897
train_width       2.642244
test_coverage     0.110156
test_width        3.494089
drugs_weight      0.044896
exit_iter         0.699206
time_taken        0.411246
dtype: float64

In [111]:
check_std = check.std().reset_index()

In [112]:
check_std.to_csv("Std_Metrics_" + string +  ".csv", index = False)

In [113]:
check.std()/10

train_mae         0.023918
test_mae          0.041001
train_coverage    0.006790
train_width       0.264224
test_coverage     0.011016
test_width        0.349409
drugs_weight      0.004490
exit_iter         0.069921
time_taken        0.041125
dtype: float64

In [114]:
# true_preds = []
# enkf_preds = []
# for i in range(0,reps):
#     all_preds = catch_coverages[i][-1]
#     all_preds = all_preds.mean(0)
#     catch_test_probs = expit(catch1[i][2])
#     true_probs = catch_test_probs.ravel().tolist()
#     true_preds.append(true_probs)
#     enkf_preds.append(all_preds.ravel().tolist())
#     # plt.scatter(true_probs, all_preds.ravel().tolist())
#     # plt.show()

In [115]:
# true_preds = [inner for item in true_preds for inner in item]

In [116]:
# enkf_preds = [inner for item in enkf_preds for inner in item]

In [117]:
# np.mean(np.abs(np.array(true_preds)-np.array(enkf_preds)))

In [118]:
# plt.scatter(enkf_preds,true_preds, alpha=0.5)
# plt.xlabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.ylabel("LSTM Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks( fontsize = 10, fontweight = "bold")
# plt.yticks( fontsize = 10, fontweight = "bold")
# plt.axline((0, 0), slope=1)
# plt.show()

In [119]:
# cur_idx = 0

In [120]:
# catch_preds = []
# for i in range(0, catch_coverages[cur_idx][-1].shape[1]):
#     enkf_preds = catch_coverages[cur_idx][-1][:,i,:]
#     enkf_preds_df = pd.DataFrame(enkf_preds)
#     enkf_preds_df["Test_Sample_ID"] = i 
#     enkf_preds_df.columns = ["EnKF_Preds", "Test_Sample_ID"]
#     catch_preds.append(enkf_preds_df)

In [121]:
# catch_preds_df = pd.concat(catch_preds)

In [122]:
# import seaborn as sns

In [123]:
# filtered_df = expit(catch1[cur_idx][2])

In [124]:
# # plt.figure(figsize = (10,10))
# fig = sns.boxplot(catch_preds_df, x="Test_Sample_ID", y="EnKF_Preds", showfliers=False)
# plt.scatter(range(0, filtered_df.shape[0]), filtered_df, c = "black", s = 50)
# plt.xlabel("Test Sample ID", fontsize = 15, fontweight = "bold")
# plt.ylabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks(fontsize = 10, fontweight = "bold")
# plt.yticks(fontsize = 10, fontweight = "bold")
# plt.show()

In [125]:
# ids = [i for i in range(0, filtered_df.shape[0]) if filtered_df[i] < 0.5]
# ids_more = [i for i in range(0, filtered_df.shape[0]) if filtered_df[i] > 0.5]

In [126]:
# catch_preds_df.groupby("Test_Sample_ID").quantile(q = [0.025, 0.975])

In [127]:
# catch_preds_df_filtered = catch_preds_df[catch_preds_df["Test_Sample_ID"].isin(ids_more)]

In [128]:
# filtered_true = filtered_df[ids_more,:]

In [129]:
# # plt.figure(figsize = (10,10))
# fig = sns.boxplot(catch_preds_df_filtered, x="Test_Sample_ID", y="EnKF_Preds", showfliers=False)
# plt.scatter(range(0, filtered_true.shape[0]), filtered_true, c = "black", s = 50)
# plt.xlabel("Test Sample ID", fontsize = 15, fontweight = "bold")
# plt.ylabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks(fontsize = 10, fontweight = "bold")
# plt.yticks(fontsize = 10, fontweight = "bold")
# plt.show()

In [130]:
# catch_preds_df_filtered = catch_preds_df[catch_preds_df["Test_Sample_ID"].isin(ids)]
# filtered_true = filtered_df[ids,:]
# # plt.figure(figsize = (10,10))
# fig = sns.boxplot(catch_preds_df_filtered, x="Test_Sample_ID", y="EnKF_Preds", showfliers=False)
# plt.scatter(range(0, filtered_true.shape[0]), filtered_true, c = "black")
# plt.xlabel("Test Sample ID", fontsize = 15, fontweight = "bold")
# plt.ylabel("EnKF Predictions", fontsize = 15, fontweight = "bold")
# plt.xticks(fontsize = 10, fontweight = "bold")
# plt.yticks(fontsize = 10, fontweight = "bold")
# plt.show()
# # plt.ylim((0.9,1.0))